In [2]:
import tweepy
import csv
import regex as re
import emoji
import pandas as pd

In [3]:
API_key = "ioitFb94uRpQ7Fb8BBwhJY1tV"
Secret_key = "efPYENIZH0kM7nL5wACtxKCwfCiUBLME8b3r1owrf546u1kdZG"
Bearer_token = "AAAAAAAAAAAAAAAAAAAAALdSWgEAAAAAy2gWysFURK0ya2LlPJoF6Jyb420%3DRD7FwDj05MLkdUySLhtVr0UmuABgzjsznDAzUhRadU3ewXMls1"

In [4]:
Access_token = "1097951470715052033-wovvuLKe00EM66WLeh1OBZqWbYWXAa"
Access_secret = "O13PdjF3x2rsuoFMgNdVbyGzeDvnfgRDJH7eU6RYvlX7w"

In [5]:
filepath = "covid19-data-master/covid_lies.csv"

In [6]:
def extract_emojis(s):
    return ''.join(c for c in s if c not in emoji.UNICODE_EMOJI['en']).replace('\u200d', '')

In [7]:
def extract_mentions(s):
    return ' '.join(c for c in s.split() if c[0] is not '@' and 'https:' not in c)

In [8]:
df = pd.read_csv(filepath)

In [9]:
df

,misconception_id,misconception,tweet_id,label
0,3,Coronavirus is genetically engineered.,1233965490948591616,na
1,30,Blowing conch shells destroys coronavirus pote...,1233907923765559296,na
2,57,Swans and dolphins swimming in Venice canals f...,1233911842910720000,na
3,22,Cocaine cures coronavirus.,1233947734094290944,na
4,32,Observing janata curfew will result in the red...,1233937085297332224,na
...,...,...,...,...
6586,50,It is safe for individuals infected with COVID...,1233949547715584000,na
6587,19,Vitamin C cures coronavirus.,1233966049097285632,na
6588,22,Cocaine cures coronavirus.,1233965616316334080,na
6589,61,Lions were freed to keep people off the street...,1233906211310825472,na


In [10]:
miscons = set(list(df['misconception'].values))
counts = {value: list(df['misconception'].values).count(value) for value in miscons}


In [11]:
counts

{'Hand sanitiser sold commercially does not destroy the coronavirus.': 100,
 'The first person infected is a researcher named Huang Yanling.': 100,
 'There will be 500 deaths at the end of the pandemic.': 100,
 'Vitamin C can prevent coronavirus circulation.': 100,
 'Boiled ginger can cure coronavirus.': 100,
 'Vitamin D cures coronavirus.': 100,
 'A Malabar civet was spotted walking the streets of Meppayur, India following COVID-19 lockdown': 100,
 'Whiskey and honey cure coronavirus.': 99,
 'Kenneth Copeland on Victory Channel can cure COVID-19 through the TV.': 99,
 'A decade-old map published by the World Population Project represents the 2020 coronavirus outbreak.': 99,
 'There is an "anti-coronavirus" mattress that could fight the coronavirus.': 100,
 'A person can tell if they have coronavirus or not simply by holding their breath.': 100,
 'Bank of England £20 banknotes contain a picture of a 5G mast and the SARS-CoV-2 virus.': 100,
 'There were more than 50000 cremations in Wuh

In [18]:
for _, row in df[df.misconception.eq('Salt water protects from coronavirus.')].iterrows():
    print(row['tweet_id'])

1233923349740429312
1233914008928911360
1233943337943482368
1233943646501773312
1233941746117156864
1233978600249970688
1233928616070799360
1233942062099243008
1233987552991940608
1233990728776376320
1233934099125063680
1233956470670200832
1233953574909931520
1233912908314292224
1233919191922569216
1233952229217927168
1233939028027826176
1233928635490488320
1233974605217488896
1233928459044478976
1233928756538068992
1233926153116749824
1233929510543749120
1233923171100684288
1233957761932644352
1233993330977124352
1233906478597201920
1233951542367019008
1233940535854821376
1233933116349329408
1233914894229131264
1233997748292091904
1233968325325025280
1233963107547897856
1233981074700587008
1233927653457092608
1233993662952067072
1233915761170747392
1233985566779281408
1233947738150060032
1233945278949904384
1233988613077114880
1233949107049369600
1233931749807316992
1233988126814633984
1233981373158973440
1233980898049003520
1233952566981070848
1233911463980388352
1233960501224173568


In [19]:
auth = tweepy.OAuthHandler(API_key, Secret_key)
try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepyException:
    print('Error! Failed to get request token.')
api = tweepy.API(auth)

In [23]:
i = 1
destination_folder = 'pos_tweets/'
extension = '.txt'

for con in miscons:
    print('Now on ', con)
    destination_filepath = destination_folder + str(con) + extension    
    i += 1
    
    successes = 0
    to_scrape = df[df.misconception.eq(con)]
    
    try:
        f = open(destination_filepath, 'w', encoding='UTF-8')
    except OSError:
        continue
    
    for _, row in to_scrape.iterrows():
        if row['label'] == 'pos':
            #print('yas!')
            tweet_id = row['tweet_id']

            try:
                    status = api.get_status(tweet_id, tweet_mode = 'extended')._json
                    unprocessed_text = status['full_text']
                    emojiless_text = extract_emojis(unprocessed_text)
                    final_text = extract_mentions(emojiless_text)
                    ID = status['id']
    #                     print(final_text)
    #                     print(ID)
                    f.write(final_text)
                    f.write('\n')
                    successes += 1
            except tweepy.TweepyException:
                continue

    print('wrote %d files!' %(successes))
        
    f.close()
    

Now on  Hand sanitiser sold commercially does not destroy the coronavirus.
wrote 3 files!
Now on  The first person infected is a researcher named Huang Yanling.
wrote 0 files!
Now on  There will be 500 deaths at the end of the pandemic.
wrote 0 files!
Now on  Vitamin C can prevent coronavirus circulation.
wrote 1 files!
Now on  Boiled ginger can cure coronavirus.
wrote 1 files!
Now on  Vitamin D cures coronavirus.
wrote 0 files!
Now on  A Malabar civet was spotted walking the streets of Meppayur, India following COVID-19 lockdown
wrote 0 files!
Now on  Whiskey and honey cure coronavirus.
wrote 0 files!
Now on  Kenneth Copeland on Victory Channel can cure COVID-19 through the TV.
wrote 0 files!
Now on  A decade-old map published by the World Population Project represents the 2020 coronavirus outbreak.
wrote 0 files!
Now on  There is an "anti-coronavirus" mattress that could fight the coronavirus.
Now on  A person can tell if they have coronavirus or not simply by holding their breath.
w

In [12]:
len(counts)

63